In [1]:
Threads.nthreads()

1

In [3]:
f(x) = sin(x)

f (generic function with 1 method)

In [4]:
x = randn(100000000)

100000000-element Vector{Float64}:
  0.6488815218670269
 -0.38905042331697814
  1.4113533889757908
  0.6293867097580864
 -0.4965669693794891
 -0.18951588049230653
  0.02251079629401766
 -1.688288353986386
  0.9979018708880366
  0.6860993643250246
  ⋮
  1.7176179792770403
  0.5861549943154972
  1.5844221997479975
 -1.3421998667134407
 -0.24091986519785483
 -1.2250387660791364
 -0.17431680682853287
 -0.6718145936834968
  2.865945481034396

In [6]:
@time f1 = f.(x);

  1.016487 seconds (3 allocations: 762.940 MiB, 2.04% gc time)


In [7]:
function test_threads(f,x)
    fval = similar(x)
    Threads.@threads for i = 1:length(x)
        fval[i] = f(x[i])
    end
    return fval
end

test_threads (generic function with 1 method)

In [9]:
@time f2 = test_threads(f,x);

  0.408296 seconds (25 allocations: 762.942 MiB)


In [10]:
using Distributed
addprocs()
@everywhere f(x) = sin(x)
using SharedArrays

In [11]:
function test_process(f,x)
    n = length(x)
    fval = SharedArray{Float64}(n)
    @distributed for i = 1:n
        fval[i] = f(x[i])
    end
    return fval
end
# fval

test_process (generic function with 1 method)

In [17]:
@time f4 = test_process(f,x);

  0.001549 seconds (349 allocations: 20.352 KiB)


In [18]:
using LinearAlgebra
norm(f1 - f4)

0.0